# Dummy Cl's

In [1]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)

sys.path.insert(1, '/home/zcapjru/PhD/DEScls')

from xcell.mappers import MapperDummy
from astropy.io import fits
from astropy.table import Table
import pyccl as ccl
import numpy as np
import pylab as plt
import pymaster as nmt
import healpy as hp

In [2]:
# Initialize binning scheme with 20 ells per bandpower
nside =  32
nside_mask = 32
#bands = nmt.NmtBin.from_nside_linear(nside, 20)
#ell_arr = bands.get_effective_ells()

In [3]:
ells = []
i = 11.5
while i <= 3 * nside:
    ells.append(round(i))
    #i = i*(1+i/(3 * nside))
    i = i+20*(1+i/240)
    
if ells[-1] <  3 * nside -1:   
    ells.append(3 * nside -1)
    
bands = nmt.NmtBin.from_edges(ells[:-1], ells[1:])
ell_arr = bands.get_effective_ells()

In [4]:
bands = nmt.NmtBin.from_edges(ells[:-1], ells[1:])
ell_arr = bands.get_effective_ells()

In [12]:
config = {'nside': 32, 
          'seed': 0}
dummy_maps = MapperDummy(config)

In [13]:
def compute_cl(f_a, f_b, wsp):
    # Compute the power spectrum (a la anafast) of the masked fields
    # Note that we only use n_iter=0 here to speed up the computation,
    # but the default value of 3 is recommended in general.
    cl_coupled = nmt.compute_coupled_cell(f_a, f_b)
    # Decouple power spectrum into bandpowers inverting the coupling matrix
    #cl_decoupled = wsp.decouple_cell(cl_coupled) #removed bias here

    return cl_coupled

In [44]:
dummy_maps.seed = 0

In [50]:
data = np.array(dummy_maps.get_signal_map())
data = np.loadtxt('/home/zcapjru/PhD/DEScls/xcell/tests/data/dummy_signal_s0.txt')

In [51]:
cols = fits.ColDefs([fits.Column(name='signal_map', format='D', array=data)])
hdu = fits.BinTableHDU.from_columns(cols)
hdu.writeto('dummy_signal_s0.fits', overwrite = True)

In [52]:
Table.read('dummy_signal_s0.fits')['signal_map'].data

array([ 1276.46029867, -1365.97206931, -5930.48124875, ...,
        5693.97998859, -9124.73434355,  -569.84985737])

In [53]:
np.loadtxt('/home/zcapjru/PhD/DEScls/xcell/tests/data/dummy_signal_s0.txt')

array([ 1276.46029867, -1365.97206931, -5930.48124875, ...,
        5693.97998859, -9124.73434355,  -569.84985737])

In [7]:
dummy_f =  dummy_maps.get_nmt_field()

/home/zcapjru/anaconda3/envs/PhD/lib/python3.8/site-packages/healpy/sphtfunc.py:433: FutureChangeWarning: The order of the input cl's will change in a future release.
Use new=True keyword to start using the new order.
See documentation of healpy.synalm.
  warnings.warn(
/home/zcapjru/anaconda3/envs/PhD/lib/python3.8/site-packages/healpy/sphtfunc.py:822: UserWarning: Sigma is 0.000000 arcmin (0.000000 rad) 
  warnings.warn(
/home/zcapjru/anaconda3/envs/PhD/lib/python3.8/site-packages/healpy/sphtfunc.py:827: UserWarning: -> fwhm is 0.000000 arcmin
  warnings.warn(


In [8]:
dummy_w = nmt.NmtWorkspace()
dummy_w.compute_coupling_matrix(dummy_f, dummy_f, bands)

In [9]:
dummy_cl = compute_cl(dummy_f, dummy_f, dummy_w)
dummy_nl = dummy_maps.get_nl_coupled() 

In [10]:
dummy_cl = dummy_w.decouple_cell(dummy_cl) - dummy_w.decouple_cell(dummy_nl)

ValueError: Input power spectrum has wrong shape

In [ ]:
plt.plot(ell_arr, dummy_cl[0], 'o-', label='dummy')
#plt.loglog()
plt.title('Dummy')
plt.xlabel('$\\ell$', fontsize=16)
plt.ylabel('$C_\\ell$', fontsize=16)
plt.legend(loc='lower left', ncol=2, labelspacing=0.1)
plt.show()